# Dataset Generation

Dataset for the process $pp \rightarrow hh \rightarrow b\overline{b}WW*$. The process done before this is the generation of process using the predefined run card in MadGraph followed by default Pythia8 and finally ending with Delphes Simulator result, where modifications are made in the delphes card itself.

In [1]:
# Imports pyRoot, which requires a manual compile as defaut ROOT binary comes with a python2 support
import ROOT
from ROOT import TLorentzVector
from math import sqrt

Welcome to JupyROOT 6.24/00


## Compile Instructions

Get the latest package by pulling the Git Repository
> git clone http://root.cern.ch/git/root.git

Then checkout the latest stable branch (6.22 at the time of writing)
> git checkout -b v6-22-00 v6-22-00

That is the source folder, which is renamed to root_src for differenciation, and two folders for building and installation are genrated
> mv root root_src

> mkdir root_install root_build

Then, go to the build folder for compilation
> cmake -DCMAKE_INSTALL_PREFIX=../root_install -DGSL_DIR="/usr/local" -DPYTHON_EXECUTABLE="/usr/bin/python3" -DPYTHON_INCLUDE_DIR="/usr/include/python3.8/" -DPYTHON_LIBRARY="/usr/lib/python3.8/" -Dbuiltin_xrootd="ON" -Dpython3="ON" -Droofit="ON" ../root_src

> cmake --build . --target install -j 8

Then wait till the compilation is over.

To call root as a command, source the thisroot.sh file, which can be done as follows :
> source ./root_install/bin/thisroot.sh

Or to let it be done everytime,
> cat ~/.bashrc << source thisroot.sh



## Sourcing all the libraries

The bashrc should contain the following

> export ROOTSYS=/home/blizzard/Applications/root/root_install
 
> export PATH=`$PATH:$ROOTSYS/bin`
 
> export LD_LIBRARY_PATH=`$LD_LIBRARY_PATH:$ROOTSYS/lib`


> export LD_LIBRARY_PATH=`/home/blizzard/Applications/madGraph/Delphes:$LD_LIBRARY_PATH`

> export ROOT_INCLUDE_PATH=`/home/blizzard/Applications/madgraph/Delphes/external/`


> export ROOT_INCLUDE_PATH=`/home/blizzard/Applications/madGraph/Delphes/classes/:$ROOT_INCLUDE_PATH`

> export PYTHONPATH=`$PYTHONPATH:$ROOT_INCLUDE_PATH`


In [16]:
# For importing this correctly, the bashrc needs to have the correct imports as given above
ROOT.gSystem.Load("libDelphes.so")
myfile = ROOT.TFile("/home/blizzard/Tests/bbWW/hh_10k/Events/run_01/tag_1_delphes_events.root")

## The n-tuple

The required n-tuple for the machine learning algorithm to work is <photon,lepton,jet,MET,pt,E,m>, where the first four items are identifiers for the respective particles

## Root tree and their information

The required information on all the root tree is given in the link :  https://cp3.irmp.ucl.ac.be/projects/delphes/wiki/WorkBook/RootTreeDescription

## The Azimuthal Angle

The angular distance between two nodes is to mapped in a different table, but the text later goes on to describe it as "difference in azimuthal angle is encoded in edge weights", but in the original explaination links the following equation :

$ \Delta R_{ij} = \sqrt{(\Delta \Phi_{ij})^2 + (\Delta \eta_{ij})^2} $

In [17]:
# The hyperparameters

isRapid = True           # Definition of the table
totalEvents = 10000      # Total number of Events in the dataset

In [15]:
directory = str("/home/blizzard/Tests/bbWW/hh_10k/Events/run_01/tag_1_delphes_events.root")

File = ROOT.TChain("Delphes;1")
File.Add(directory)
totalEvents = File.GetEntries()

eventDict = []
for i in range(totalEvents):
	Entry = File.GetEntry(i)

	epArray = []
	ezArray = []
	az_angle = [] # Azimuthal Angle
	ra_angle = [] # Rapidity

	EntryFromBranch = File.Photon.GetEntries()	
	for j in range(EntryFromBranch):
		particleArray = [1,0,0,0,File.GetLeaf("Photon.PT").GetValue(j),File.GetLeaf("Photon.E").GetValue(j),0]
		epArray.append(particleArray)
		
		az_angle.append(File.GetLeaf("Photon.Phi").GetValue(j))
		ra_angle.append(File.GetLeaf("Photon.Eta").GetValue(j))
		

	EntryFromBranch = File.Jet.GetEntries()
	for j in range(EntryFromBranch):

		Jet = TLorentzVector()
		Jet.SetPtEtaPhiM(File.GetLeaf("Jet.PT").GetValue(j),File.GetLeaf("Jet.Eta").GetValue(j),File.GetLeaf("Jet.Phi").GetValue(j),File.GetLeaf("Jet.Mass").GetValue(j))

		particleArray = [0,0,1 if int(File.GetLeaf("Jet.BTag").GetValue(j)) == 1 else -1,0,File.GetLeaf("Jet.PT").GetValue(j),Jet.Energy(),File.GetLeaf("Jet.Mass").GetValue(j)]
		epArray.append(particleArray)

		az_angle.append(File.GetLeaf("Jet.Phi").GetValue(j))
		ra_angle.append(File.GetLeaf("Jet.Eta").GetValue(j))

	EntryFromBranch = File.Electron.GetEntries()
	for j in range(EntryFromBranch):
		Electron = TLorentzVector()
		Electron.SetPtEtaPhiM(File.GetLeaf("Electron.PT").GetValue(j),File.GetLeaf("Electron.Eta").GetValue(j),File.GetLeaf("Electron.Phi").GetValue(j),0)

		particleArray = [0,int(File.GetLeaf("Electron.Charge").GetValue(j)),0,0,File.GetLeaf("Electron.PT").GetValue(j),Electron.Energy(),0]
		epArray.append(particleArray)
		
		az_angle.append(File.GetLeaf("Electron.Phi").GetValue(j))
		ra_angle.append(File.GetLeaf("Electron.Eta").GetValue(j))


	EntryFromBranch = File.MissingET.GetEntries()
	for j in range(EntryFromBranch):
		particleArray = [0,0,0,1,File.GetLeaf("MissingET.MET").GetValue(j),File.GetLeaf("MissingET.MET").GetValue(j),0]	
		epArray.append(particleArray)

		az_angle.append(File.GetLeaf("MissingET.Phi").GetValue(j))
		ra_angle.append(File.GetLeaf("MissingET.Eta").GetValue(j))


	EntryFromBranch = File.Muon.GetEntries()
	for j in range(EntryFromBranch):
		
		Muon = TLorentzVector()
		Muon.SetPtEtaPhiM(File.GetLeaf("Muon.PT").GetValue(j),File.GetLeaf("Muon.Eta").GetValue(j),File.GetLeaf("Muon.Phi").GetValue(j),0)
		
		particleArray = [0,int(File.GetLeaf("Muon.Charge").GetValue(j)),0,0,File.GetLeaf("Muon.PT").GetValue(j),Muon.Energy(),0]
		epArray.append(particleArray)
		
		az_angle.append(File.GetLeaf("Muon.Phi").GetValue(j))
		ra_angle.append(File.GetLeaf("Muon.Eta").GetValue(j))
		

	# Getting the Angular Angle Distance
	noPart = len(az_angle) # Number of particles in the event

	if isRapid:
		for i in range(noPart):
			tempAng = []
			for j in range(noPart):
				tempAng.append(sqrt((az_angle[i] - az_angle[j])**2 + (ra_angle[i] - ra_angle[j])**2))
			ezArray.append(tempAng)

	else:
		for i in range(noPart):
			tempAng = []
			for j in range(noPart):
				tempAng.append(az_angle[i] - az_angle[j])
			ezArray.append(tempAng)

	eventDict.append({"fourMomenta" : epArray,"azimuthalAngle" : ezArray,
                              "phiList" : az_angle,"etaList" : ra_angle})

## Issues

These are issues faced by me currently in the process of dataset generation:
- The mass of leptons is not available in the root tree ($m_{ll}$)
- The Energy of leptons, jets and MET is not available

Possbile Fixes/Updates
- Online source claims that lepton enrgy is taken to be zero (not in the dataset though)
- For MET, the Energy and the PT is the same

## Sample Record

Details of a sample record shown for demonstration

In [18]:
from numpy.random import randint
record = randint(0,10000)

In [19]:
print(" Photon | Lepton | Jet  | MET |  pt   |   E   |  mass ")
print("------------------------------------------------------")
for i in eventDict[record]["fourMomenta"]:
    print("   "+ str(i[0]) + "    |" + "   " +str('%2d' % i[1]) + "   |" + "  "+str('%2d'%i[2])+"  |" + "  "+ str(i[3]) + "  |" + str('%7.3f' % i[4]) + "|" + str('%7.3f' % i[5]) + "|" + str('%7.3f' % i[6]))

 Photon | Lepton | Jet  | MET |  pt   |   E   |  mass 
------------------------------------------------------
   0    |    0   |  -1  |  0  |135.462|137.829| 13.709
   0    |    0   |   1  |  0  | 58.324|225.508|  9.179
   0    |    0   |  -1  |  0  | 58.169|125.806|  4.549
   0    |    0   |   1  |  0  | 29.502| 37.244|  4.574
   0    |    0   |   0  |  1  | 64.241| 64.241|  0.000
   0    |    1   |   0  |  0  | 71.555|138.901|  0.000


In [20]:
partNo = len(eventDict[record]["azimuthalAngle"])

print(" "*10 + "|",end="")
for i in range(partNo):
    print('%5d'%i + "     |",end="")
print()
print("-----------"*(partNo+1))

for i in range(partNo):
    print('%5d'%i + "     |",end="")
    for j in range(partNo):
        print('%10.5f'%eventDict[record]["azimuthalAngle"][i][j] + "|",end="")
    print()
        

          |    0     |    1     |    2     |    3     |    4     |    5     |
-----------------------------------------------------------------------------
    0     |   0.00000|   2.78618|   1.54378|   2.35286|   4.23584|   3.06858|
    1     |   2.78618|   0.00000|   3.03853|   4.55431|   5.28456|   4.97653|
    2     |   1.54378|   3.03853|   0.00000|   1.55254|   5.77954|   1.95013|
    3     |   2.35286|   4.55431|   1.55254|   0.00000|   6.22495|   0.81344|
    4     |   4.23584|   5.28456|   5.77954|   6.22495|   0.00000|   7.03326|
    5     |   3.06858|   4.97653|   1.95013|   0.81344|   7.03326|   0.00000|


## Exporting the Dataset

Exporting the dataset as a hdf5 due to the dimensionality of the dataset.

$\dim(x) = \mathcal{R}^2$ <br>
$\dim(X) = \mathcal{R}^3$

In [21]:
import h5py
import numpy as np
import awkward as ak
import pandas as pd
import json

In [22]:
xArray = []
for i in range(totalEvents):
    xArray.append(eventDict[i]["fourMomenta"])

azArray = []
for i in range(totalEvents):
    azArray.append(eventDict[i]["azimuthalAngle"])

eta = []
for i in range(totalEvents):
    eta.append(eventDict[i]["etaList"])

phi = []
for i in range(totalEvents):
    phi.append(eventDict[i]["phiList"])

In [95]:
hf = h5py.File('MPNN_Dataset.h5','w')
partArray = hf.create_group("ParticleArray")
azimuthalArray = hf.create_group("AzimuthalAngle")
etaArray = hf.create_group("EtaAngle")
phiArray = hf.create_group("PhiAngle")

In [96]:
# Convert Particle Array to HDF5 Group

ak_array = ak.from_iter(xArray)
form, length, container = ak.to_buffers(ak_array,container=partArray)
partArray.attrs["form"] = form.tojson()
partArray.attrs["length"] = json.dumps(length)
partArray.keys()

<KeysViewHDF5 ['part0-node0-offsets', 'part0-node1-offsets', 'part0-node2-data']>

In [97]:
# Convert Azimuthal Angle Array to HDF5 Group

ak_array = ak.from_iter(azArray)
form, length, container = ak.to_buffers(ak_array,container=azimuthalArray)
azimuthalArray.attrs["form"] = form.tojson()
azimuthalArray.attrs["length"] = json.dumps(length)
azimuthalArray.keys()

<KeysViewHDF5 ['part0-node0-offsets', 'part0-node1-offsets', 'part0-node2-data']>

In [98]:
ak_array = ak.from_iter(eta)
form, length, container = ak.to_buffers(ak_array,container=etaArray)
etaArray.attrs["form"] = form.tojson()
etaArray.attrs["length"] = json.dumps(length)
etaArray.keys()

<KeysViewHDF5 ['part0-node0-offsets', 'part0-node1-data']>

In [99]:
ak_array = ak.from_iter(phi)
form, length, container = ak.to_buffers(ak_array,container=phiArray)
phiArray.attrs["form"] = form.tojson()
phiArray.attrs["length"] = json.dumps(length)
phiArray.keys()

<KeysViewHDF5 ['part0-node0-offsets', 'part0-node1-data']>

In [104]:
hf.close()

## Accesor

Creating an accesor that converts the generated HDF5 file to python list for demonstration

In [111]:
hf = h5py.File('../../datasets/partonCuts/ttbar_10k.h5','r')
print(hf.keys())
partArray = hf.get("ParticleArray")
azimuthalArray = hf.get("AzimuthalAngle")
etaArray = hf.get("EtaAngle")
phiArray = hf.get("PhiAngle")

<KeysViewHDF5 ['AzimuthalAngle', 'EtaAngle', 'ParticleArray', 'PhiAngle']>


In [112]:
etaArray.attrs.keys()

<KeysViewHDF5 ['form', 'length']>

In [113]:
reconstitutedPartArray = ak.from_buffers(
    ak.forms.Form.fromjson(partArray.attrs["form"]),
    json.loads(partArray.attrs["length"]),
    {k: np.asarray(v) for k, v in partArray.items()},
)
reconstitutedPartArray

reconstitutedAzAngle = ak.from_buffers(
    ak.forms.Form.fromjson(azimuthalArray.attrs["form"]),
    json.loads(azimuthalArray.attrs["length"]),
    {k: np.asarray(v) for k, v in azimuthalArray.items()},
)
reconstitutedAzAngle

reconstitutedEtaAngle = ak.from_buffers(
    ak.forms.Form.fromjson(etaArray.attrs["form"]),
    json.loads(etaArray.attrs["length"]),
    {k: np.asarray(v) for k, v in etaArray.items()},
)
reconstitutedEtaAngle

reconstitutedPhiAngle = ak.from_buffers(
    ak.forms.Form.fromjson(phiArray.attrs["form"]),
    json.loads(phiArray.attrs["length"]),
    {k: np.asarray(v) for k, v in phiArray.items()},
)
reconstitutedEtaAngle

<Array [[1.03, 1.21, 0.996, ... -1.38, 0.391]] type='10000 * var * float64'>

In [114]:
particleArray = ak.to_list(reconstitutedPartArray)
azArray = ak.to_list(reconstitutedAzAngle)
etaArray = ak.to_list(reconstitutedEtaAngle)
phiArray = ak.to_list(reconstitutedPhiAngle)

In [115]:
print(len(particleArray),len(particleArray[0]),len(particleArray[0][0]))

10000 7 7


In [116]:
print(len(etaArray),len(etaArray[0]))

10000 7


## Baseline Cuts

Applying the baseline cuts as required in the paper

In [8]:
def printPretty(record):
    print(" Photon | Lepton | Jet  | MET |  pt   |   E   |  mass ")
    print("------------------------------------------------------")
    for i in record:
        print("   "+ str(i[0]) + "    |" + "   " +str('%2d' % i[1]) + "   |" + "  "+str('%2d'%i[2])+"  |" + "  "+ str(i[3]) + "  |" + str('%7.3f' % i[4]) + "|" + str('%7.3f' % i[5]) + "|" + str('%7.3f' % i[6]))


In [14]:
# Cuts for Leading Jet being b-tagged
# Leading Jets are Higher PT Jets

tempDictList = eventDict
cutDict = []
for i in tempDictList:
    
    noJets = 0
    ptJetsb = []
    ptJetsNob = []
    ptAllJets = []
    
    for momenta in i["fourMomenta"]:
        if momenta[2] == 1:
            noJets += 1
            ptJetsb.append(momenta[4])
            ptAllJets.append(momenta[4])
            
        elif momenta[2] == -1:
            noJets += 1
            ptJetsNob.append(momenta[4])
            ptAllJets.append(momenta[4])
    
    if len(ptJetsb) < 2:
        print()
        print("No 2 b-tagged Jets")
        printPretty(i["fourMomenta"])
        eventDict.remove(i)
        totalEvents -= 1
        continue

    flag = False
    ptAllJets.sort(reverse=True)
    
    if ptAllJets[0] in ptJetsb and ptAllJets[1] in ptJetsb:
        flag = True
    
    if flag == False:
        print()
        print("Non b-tagged")
        printPretty(i["fourMomenta"])
        eventDict.remove(i)
        totalEvents -= 1
        continue

    cutDict.append(i)
    
    #self.printPretty("fourMomenta",i["fourMomenta"])

 7.954
   0    |    0   |  -1  |  0  | 32.036|203.869|  1.123
   0    |    0   |   0  |  1  | 92.986| 92.986|  0.000
   0    |    1   |   0  |  0  | 18.628| 65.892|  0.000

Non b-tagged
 Photon | Lepton | Jet  | MET |  pt   |   E   |  mass 
------------------------------------------------------
   0    |    0   |   1  |  0  | 48.327| 50.823|  6.545
   0    |    0   |  -1  |  0  | 43.432| 79.111|  8.864
   0    |    0   |   1  |  0  | 39.115|122.564|  5.846
   0    |   -1   |   0  |  0  | 28.382| 30.668|  0.000
   0    |    0   |   0  |  1  | 15.884| 15.884|  0.000
   0    |    1   |   0  |  0  | 32.319| 34.813|  0.000

Non b-tagged
 Photon | Lepton | Jet  | MET |  pt   |   E   |  mass 
------------------------------------------------------
   0    |    0   |  -1  |  0  |244.642|454.351| 16.433
   0    |    0   |   1  |  0  |128.456|137.071| 13.980
   0    |    0   |  -1  |  0  |109.071|348.924| 13.952
   0    |    0   |   1  |  0  |108.193|180.150| 20.426
   0    |    0   |  -1  |  0  

In [15]:
for record in range(len(cutDict)):
    print(" Photon | Lepton | Jet  | MET |  pt   |   E   |  mass ")
    print("------------------------------------------------------")
    for i in cutDict[record]["fourMomenta"]:
        print("   "+ str(i[0]) + "    |" + "   " +str('%2d' % i[1]) + "   |" + "  "+str('%2d'%i[2])+"  |" + "  "+ str(i[3]) + "  |" + str('%7.3f' % i[4]) + "|" + str('%7.3f' % i[5]) + "|" + str('%7.3f' % i[6]))

046| 62.046|  0.000
   0    |    1   |   0  |  0  | 60.490| 73.361|  0.000
   0    |   -1   |   0  |  0  | 12.502| 12.521|  0.000
 Photon | Lepton | Jet  | MET |  pt   |   E   |  mass 
------------------------------------------------------
   0    |    0   |   1  |  0  | 62.907| 63.707|  8.563
   0    |    0   |   1  |  0  | 49.231|103.588| 12.003
   0    |    0   |  -1  |  0  | 36.716| 38.698|  3.328
   0    |    0   |  -1  |  0  | 28.849| 29.770|  6.917
   0    |    1   |   0  |  0  | 30.088| 36.482|  0.000
   0    |    0   |   0  |  1  | 37.334| 37.334|  0.000
 Photon | Lepton | Jet  | MET |  pt   |   E   |  mass 
------------------------------------------------------
   0    |    0   |   1  |  0  | 83.371|111.462| 10.857
   0    |    0   |   1  |  0  | 45.120| 75.783| 12.572
   0    |    1   |   0  |  0  | 29.695| 56.982|  0.000
   0    |    0   |   0  |  1  | 45.921| 45.921|  0.000
 Photon | Lepton | Jet  | MET |  pt   |   E   |  mass 
---------------------------------------------

In [17]:
len(cutDict)

1178

In [20]:
for i in cutDict:
    noBJets = 0
    ptBJets = []
    ptAllJets = []
    for j in i['fourMomenta']:
        if j[2] == 1:
            ptAllJets.append(j[4])
            ptBJets.append(j[4])
        elif j[2] == -1:
            ptAllJets.append(j[4])

    ptAllJets.sort(reverse=True)
    if ptAllJets[0] not in ptBJets or ptAllJets[1] not in ptBJets:
        printPretty(i['fourMomenta'])